# Explore the VectorBT Module

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import vectorbt as vbt

/tmp/ipykernel_21263/794029728.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
start = '2019-01-01 UTC'  # crypto is in UTC
end = '2020-01-01 UTC'
btc_price = vbt.YFData.download('BTC-USD', start=start, end=end).get('Close')
btc_price

/home/animalibera/.pyenv/versions/3.12.1/lib/python3.12/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


Date
2019-01-01 00:00:00+00:00    3843.520020
2019-01-02 00:00:00+00:00    3943.409424
2019-01-03 00:00:00+00:00    3836.741211
2019-01-04 00:00:00+00:00    3857.717529
2019-01-05 00:00:00+00:00    3845.194580
                                ...     
2019-12-27 00:00:00+00:00    7290.088379
2019-12-28 00:00:00+00:00    7317.990234
2019-12-29 00:00:00+00:00    7422.652832
2019-12-30 00:00:00+00:00    7292.995117
2019-12-31 00:00:00+00:00    7193.599121
Freq: D, Name: Close, Length: 365, dtype: float64

In [3]:
fast_ma = vbt.MA.run(btc_price, 10, short_name="fast")
slow_ma = vbt.MA.run(btc_price, 20, short_name="slow")

entries = fast_ma.ma_crossed_above(slow_ma)
exits = fast_ma.ma_crossed_below(slow_ma)

pf = vbt.Portfolio.from_signals(btc_price, entries, exits)

pf.total_return()

0.6351860771192923

In [5]:
fast_ma = vbt.MA.run(btc_price, [10, 20], short_name="fast")
slow_ma = vbt.MA.run(btc_price, [30, 30], short_name="slow")

entries = fast_ma.ma_crossed_above(slow_ma)
exits = fast_ma.ma_crossed_below(slow_ma)

pf = vbt.Portfolio.from_signals(btc_price, entries, exits)

pf.total_return()

fast_window  slow_window
10           30             0.847151
20           30             0.543411
Name: total_return, dtype: float64

In [6]:
eth_price = vbt.YFData.download('ETH-USD', start=start, end=end).get('Close')
comb_price = btc_price.vbt.concat(eth_price, keys=pd.Index(['BTC', 'ETH'], name='symbol'))
comb_price.vbt.drop_levels(-1, inplace=True)
comb_price

/home/animalibera/.pyenv/versions/3.12.1/lib/python3.12/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


symbol,BTC,ETH
Date,,
2019-01-01 00:00:00+00:00,3843.520020,140.819412
2019-01-02 00:00:00+00:00,3943.409424,155.047684
2019-01-03 00:00:00+00:00,3836.741211,149.135010
2019-01-04 00:00:00+00:00,3857.717529,154.581940
2019-01-05 00:00:00+00:00,3845.194580,155.638596
...,...,...
2019-12-27 00:00:00+00:00,7290.088379,127.214607
2019-12-28 00:00:00+00:00,7317.990234,128.322708
2019-12-29 00:00:00+00:00,7422.652832,134.757980


In [7]:
fast_ma = vbt.MA.run(comb_price, [10, 20], short_name='fast')
slow_ma = vbt.MA.run(comb_price, [30, 30], short_name='slow')

entries = fast_ma.ma_crossed_above(slow_ma)
exits = fast_ma.ma_crossed_below(slow_ma)

pf = vbt.Portfolio.from_signals(comb_price, entries, exits)
pf.total_return()

fast_window  slow_window  symbol
10           30           BTC       0.847151
                          ETH       0.244204
20           30           BTC       0.543411
                          ETH      -0.319102
Name: total_return, dtype: float64

In [12]:
mean_return = pf.total_return().groupby('symbol').mean()
#mean_return.vbt.barplot(xaxis_title='Symbol', yaxis_title='Mean total return')
mean_return

symbol
BTC    0.695281
ETH   -0.037449
Name: total_return, dtype: float64

In [15]:
pf.stats()


/tmp/ipykernel_21263/3978197583.py:1: UserWarning:

Object has multiple columns. Aggregating using <function mean at 0x7f1334117b00>. Pass column to select a single column/group.



Start                         2019-01-01 00:00:00+00:00
End                           2019-12-31 00:00:00+00:00
Period                                365 days 00:00:00
Start Value                                       100.0
End Value                                    132.891607
Total Return [%]                              32.891607
Benchmark Return [%]                          39.601109
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              49.092492
Max Drawdown Duration                 188 days 00:00:00
Total Trades                                       6.25
Total Closed Trades                                 6.0
Total Open Trades                                  0.25
Open Trade PnL                                -1.470394
Win Rate [%]                                       45.0
Best Trade [%]                                86.873771
Worst Trade [%]                              -21